In [1]:
import pandas as pd

from copy import deepcopy

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,roc_auc_score
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('DMC//transact_train.csv', sep=',')

In [4]:
def groupData(data, field):
    group = data[field]
    agg = data.groupby([group])
    splited = []
    for group in agg:
        splited.append(pd.DataFrame(data = group[1], columns=data.columns.values))
    return splited

In [5]:
data

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,0.00,1,59.99,59.99,59.99,1,59.99,...,?,1,600,70,21,1,43,1,49,y
1,1,6,5,11.94,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
2,1,6,5,39887.00,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
3,2,6,5,0.00,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
4,2,6,5,15633.00,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429008,49998,18,7,5988882.00,77,9.99,149.99,5253.28,3,49.95,...,?,?,?,?,?,?,?,?,?,n
429009,49999,18,7,675114.00,6,59.0,199.99,509.96,1,89.99,...,?,25038,?,?,?,0,24,1,4,n
429010,49999,18,7,715341.00,7,59.0,649.99,1159.95,1,89.99,...,completely orderable,25038,?,?,?,0,24,1,4,n
429011,50000,18,7,0.00,1,6.99,6.99,6.99,1,6.99,...,?,47,300,537,5,0,53,1,34,n


In [6]:
splited = groupData(data,"sessionNo")

In [9]:
splited[0]

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,0.00,1,59.99,59.99,59.99,1,59.99,...,?,1,600,70,21,1,43,1,49,y
1,1,6,5,11.94,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
2,1,6,5,39887.00,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y


In [10]:
def granul_fields(splited):
    bStep_count = []
    countLog = []
    modes = []
    for session in splited:
        countStep = 0
        countLog.append(len(session))
        modes.append(session['bStep'].mode()[0])
        for j in range(len(session)):
            if(session['bStep'].iloc[j] != "?" and int(session['bStep'].iloc[j]) >= 3):
                countStep +=1
        bStep_count.append(countStep)
    return bStep_count,countLog,modes

In [11]:
bStep_count,countLog,modes = granul_fields(splited)

In [14]:
modes[0]

'?'

In [18]:
#Mudando a granularidade
def granul_change(data):
    return data.groupby(['sessionNo'], as_index=False).last()

data = granul_change(data)

In [21]:
data

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,39887.0,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
1,2,6,5,94469.0,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
2,3,6,5,341613.0,11,9.99,29.99,109.95,2,9.99,...,completely orderable,3,1800,475,302,12,45,1,11,y
3,4,6,5,42812.0,4,4.99,4.99,19.96,1,4.99,...,completely not orderable,?,?,?,?,?,?,?,?,n
4,5,6,5,2816046.0,45,12.99,179.95,1093.72,4,19.99,...,completely orderable,4,800,503,18,1,46,1,40,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,18,7,7271812.0,197,6.99,59.99,4315.03,13,9.99,...,completely orderable,?,?,?,?,?,?,?,?,y
49996,49997,18,7,667219.0,8,70.99,89.99,667.88,1,89.99,...,completely orderable,?,?,?,?,?,?,?,?,n
49997,49998,18,7,5988882.0,77,9.99,149.99,5253.28,3,49.95,...,?,?,?,?,?,?,?,?,?,n
49998,49999,18,7,715341.0,7,59.0,649.99,1159.95,1,89.99,...,completely orderable,25038,?,?,?,0,24,1,4,n


In [20]:
def correlation_fields(data):
    # diffCounts
    diffCounts = (data['cCount']-data['bCount']).values

    # durCount
    durCount = (data['duration']/data['cCount'])
    durCount = durCount.replace(float("inf"), "?", regex=False).values
    # Testar depois com outros valores para tratar o inf
    return diffCounts,durCount
diffCounts,durCount = correlation_fields(data)